In [1]:
import pandas as pd

subway_delay = pd.read_csv("file:///Users/bhavanarama/Desktop/Monthly_Subway_Line_Delay_Data.csv")
weather_data = pd.read_csv("file:///Users/bhavanarama/Desktop/Monthly_Weather_Data.csv")

subway_delay = subway_delay[['Month','line', 'average_delay_time']]
weather_data = weather_data[['Month','Max_Temp', 'Min_Temp']]


In [2]:
request_line = "1"
line = "line"
mask = subway_delay['line'] == request_line
subway_line = subway_delay[mask]


In [3]:
result = pd.merge(weather_data, subway_line, on='Month')
train_data  = result[['Max_Temp', 'Min_Temp','average_delay_time']]
train_data.head()

,Max_Temp,Min_Temp,average_delay_time
0,55,8,1.27
1,43,3,1.26
2,62,13,1.11
3,77,33,0.85
4,87,43,0.90


In [4]:
import requests
import json


URL = 'http://api.openweathermap.org/data/2.5/forecast?q=Queens,us&appid=1026b7625ff1ec232a7927edbe48af1e'
r = requests.get(URL)
data = r.json()
length = len(data["list"])
maximum_temp = []
minimum_temp = []
day = 0
while day < length:
    max_temp = -999
    min_temp = 999
    for cur in range(day, day + 8):
        if cur < length:
            temp_F = (9/5*(data["list"][cur]["main"]["temp_max"] - 273.15)) + 32
            max_temp = max(max_temp,temp_F)


            temp_F1 = (9/5*(data["list"][cur]["main"]["temp_max"] - 273.15)) + 32
            min_temp = min(min_temp, temp_F1)
    maximum_temp.append(max_temp)
    minimum_temp.append(min_temp)
    day += 8
    
print(maximum_temp)
print(minimum_temp)

[46.981400000000065, 57.185600000000036, 47.96600000000001, 39.68600000000007, 42.20420000000007]
[30.59600000000005, 49.521200000000064, 37.106600000000086, 32.3852, 31.23320000000002]


In [5]:
import numpy as np
dictionary = {'Max_Temp': maximum_temp, 'Min_Temp' : minimum_temp}
test_data = pd.DataFrame(dictionary)
npMatrix = np.matrix(train_data)

X, Y = npMatrix[:,0:2], npMatrix[:,2]

In [6]:
from sklearn.linear_model import LinearRegression
Linreg = LinearRegression().fit(X,Y)
#Linreg = Ridge(alpha=1.0,fit_intercept=True,max_iter=100, solver='saga').fit(X,Y)
results = Linreg.predict(test_data)
results

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


array([[1.19349416],
       [1.21438143],
       [1.21786459],
       [1.25166784],
       [1.22903739]])

In [7]:
import math

time = []
for result in results:
    current = []
    minute = math.floor(result)
    seconds = (result * 60)%60
    time.append([minute, int(seconds)])
print(time)

[[1, 11], [1, 12], [1, 13], [1, 15], [1, 13]]
